In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
matplotlib.rcParams['figure.figsize'] = (5.0, 4.0)

In [3]:
from Preprocessor import Preprocessor
from Registerer import Registerer
import SimpleITK as sitk
import ndreg
import numpy as np
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *

In [4]:
import scipy

In [5]:
def read_affine(string, scale):
    a = np.reshape(np.array([float(i) for i in string.split()]), (4,4))
    a[:3,3] /= scale
    return a

In [6]:
def fid_apply_affine(fids, affine):
    return np.array([affine.dot(i) for i in fids])[:,:3]

#### functions to parse viz link points

In [7]:
import json
import re
def get_points_from_viz_link(link, layer):
    link_t = link[link.find('!')+1:]
    link_tr = link_t.replace("_", ",")
    link_tr = link_tr.replace("\'","\"")
    lj = json.loads(link_tr)
#     return lj
    vox_size = np.array(lj['navigation']['pose']['position']['voxelSize']) / 1.0e3 # convert nm to um
    points = np.array(lj['layers'][layer]['points'])
    return {'points': points, 'vox_size': vox_size}

def add_points_layer_to_viz_link(link, layer, points=[[]]):
    link_t = link[link.find('!')+1:]
    link_tr = link_t.replace("_", ",")
    link_tr = link_tr.replace("\'","\"")
    lj = json.loads(link_tr)
    lj['layers'][layer] = {'type':'pointAnnotation', 'points':points}\
    
    # fix some commas that need to be underscores
    for i in lj['layers'].keys():
        try:
            tmp = lj['layers'][i]['source']
            idx = tmp.find('?')
            lj['layers'][i]['source'] = tmp[:idx].replace(",","_") + tmp[idx:]
            print('replaced commas')
        except Exception as e:
            pass
            print(e.message)
    return link[:link.index('!')+1] + json.dumps(lj)
    

In [8]:
def plot_landmark_ortho(img, pt, refThreshold):
    test_img = sitk.GetArrayViewFromImage(img[:,:,pt[2]]).astype('uint16')
    plt.imshow(test_img, vmax=refThreshold, cmap='gray')
    plt.plot(pt[0], pt[1], 'ro')
    plt.colorbar()
    plt.show()
    test_img = sitk.GetArrayViewFromImage(img[:,pt[1],:]).astype('uint16')
    plt.imshow(test_img, vmax=refThreshold, cmap='gray')
    plt.plot(pt[0], pt[2], 'ro')
    plt.colorbar()
    plt.show()
    test_img = sitk.GetArrayViewFromImage(img[pt[0],:,:]).astype('uint16')
    plt.imshow(test_img, vmax=refThreshold, cmap='gray')
    plt.plot(pt[1], pt[2], 'ro')
    plt.colorbar()
    plt.show()

### Step 1: Load data

In [9]:
img = ndreg.imgRead('../forVikram/ailey-dev_thy1eyfp_control_9_upload20170919_ch1_res3.img')
atlas = ndreg.imgRead('../forVikram/average_template_50.tif')

In [10]:
atlas.SetSpacing((0.05, 0.05, 0.05))

### set some constants

In [211]:
true_size_atlas = [13200.0, 8000.0, 11400.0] # mm
true_size_raw = [13209.6, 7353.0, 11145.6] # mm
scale_size = np.array(true_size_atlas)/np.array(true_size_raw)
guess_size_raw = np.array([2160, 2560, 1425]) * 1.0 # mm
center_raw = np.array(true_size_raw)/2.0 # mm
center_atlas = np.array(true_size_atlas)/2.0 # mm
raw_fid_o = "RPI"
atlas_fid_o = "PIR"
trans_o = "PIR"
pad = 20 # voxel padding for phis, phiinvs, and x,y,z

In [12]:
scale_size

array([ 0.99927326,  1.0879913 ,  1.02282515])

In [13]:
# read inv displacement fields
phiinvx = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiinvx.img')[pad:-pad, pad:-pad, pad:-pad]
phiinvy = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiinvy.img')[pad:-pad, pad:-pad, pad:-pad]
phiinvz = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiinvz.img')[pad:-pad, pad:-pad, pad:-pad]

In [14]:
# read inv displacement fields
phix = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phix.img')[pad:-pad, pad:-pad, pad:-pad]
phiy = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiy.img')[pad:-pad, pad:-pad, pad:-pad]
phiz = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiz.img')[pad:-pad, pad:-pad, pad:-pad]
# phix = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phix.img')
# phiy = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiy.img')
# phiz = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/phiz.img')

In [67]:
# read grids
x = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/x.img')[pad:-pad, pad:-pad, pad:-pad]
y = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/y.img')[pad:-pad, pad:-pad, pad:-pad]
z = ndreg.imgRead('../ndreg/control9_for_vikram_nov_12/z.img')[pad:-pad, pad:-pad, pad:-pad]

In [16]:
t2a_s = """0.986066 -0.094286 0.024768 516.012050
0.089707 0.992495 -0.013788 -443.893828
-0.009284 0.016666 0.950780 190.920903
0.000000 0.000000 0.000000 1.000000
"""
a2t_s = """1.005173 0.095907 -0.024794 -471.375106
-0.090695 0.998663 0.016845 486.883867
0.011405 -0.016569 1.051231 -213.941982
0.000000 0.000000 0.000000 1.000000
"""

In [17]:
scale = 1.0 # convert from um to X

In [18]:
t2a = read_affine(t2a_s, scale)
a2t = read_affine(a2t_s, scale)

In [19]:
phix_np = sitk.GetArrayViewFromImage(phix)
phiy_np = sitk.GetArrayViewFromImage(phiy)
phiz_np = sitk.GetArrayViewFromImage(phiz)
phiinvx_np = sitk.GetArrayViewFromImage(phiinvx)
phiinvy_np = sitk.GetArrayViewFromImage(phiinvy)
phiinvz_np = sitk.GetArrayViewFromImage(phiinvz)

In [20]:
phi = np.zeros(phix_np.shape + (3,))
phiinv = np.zeros(phiinvx_np.shape + (3,))

In [21]:
phi[:,:,:,0] = phix_np
phi[:,:,:,1] = phiy_np
phi[:,:,:,2] = phiz_np
phiinv[:,:,:,0] = phiinvx_np
phiinv[:,:,:,1] = phiinvy_np
phiinv[:,:,:,2] = phiinvz_np

## load fiducials

In [43]:
r = Registerer(atlas, img, "ASR", "LPS")

In [44]:
afid = r._parse_fiducial_file('../Manual Fiducials/Control9/average_template_10-landmarks.fcsv', 0.01/10.0)
rfid = r._parse_fiducial_file('../Manual Fiducials/Control9/Control9Auto-landmarks.fcsv', 0.00516/1.0)
# rfid = r._parse_fiducial_file('../Manual Fiducials/Control9/Control9Auto-landmarks.fcsv', 1.0/1.0)

In [45]:
atlas_link = "https://viz.boss.neurodata.io/#!{'layers':{'average,10um':{'type':'image'_'source':'boss://https://api.boss.neurodata.io/ara_2016/sagittal_10um/average_10um?window=0,500'_'opacity':0.5_'max':0.73}_'fiducials':{'type':'pointAnnotation'_'points':[[856.4130249023438_596.4749755859375_567.3800048828125]_[429.34600830078125_385.6579895019531_567.2789916992188]_[726.6300048828125_223.68699645996094_413.37701416015625]_[743.4439697265625_220.74200439453125_726.43798828125]_[1086.510009765625_179.08900451660156_574.9819946289062]_[845.3040161132812_373.6520080566406_899.3489990234375]_[842.4249877929688_373.7869873046875_242.19097900390625]_[422.84100341796875_386.5840148925781_575.4089965820312]_[418.4840087890625_544.5599975585938_453.77301025390625]_[419.5530090332031_546.2000122070312_690.2919921875]_[1061.989990234375_108.91500091552734_481.458984375]_[1062.5999755859375_107.93299865722656_659.166015625]_[305.2640075683594_424.6080017089844_437.38897705078125]_[315.79998779296875_424.5989990234375_699.3399658203125]_[356.3429870605469_510.0050048828125_677.0899658203125]_[348.364990234375_518.1950073242188_467.072021484375]_[411.2929992675781_367.7699890136719_605.97998046875]_[413.6659851074219_372.2860107421875_542.6339721679688]_[383.4259948730469_364.40399169921875_732.3609619140625]_[386.55999755859375_388.53900146484375_422.14697265625]_[520.260009765625_542.1240234375_571.7949829101562]_[607.6909790039062_245.38400268554688_496.03900146484375]_[605.8629760742188_239.53900146484375_645.1309814453125]_[643.8519897460938_263.40899658203125_602.1900024414062]_[634.6740112304688_254.8509979248047_541.1610107421875]_[717.6199951171875_288.0530090332031_273.52899169921875]_[724.35400390625_307.9209899902344_870.052978515625]_[1052.530029296875_253.63400268554688_574.5969848632812]_[1094.3499755859375_352.6419982910156_418.760009765625]_[1090.8699951171875_322.1719970703125_711.47998046875]_[1104.2099609375_318.40399169921875_192.0989990234375]_[1047.1700439453125_687.208984375_702.7760009765625]_[1048.260009765625_687.0330200195312_442.9110107421875]_[1196.7099609375_311.8699951171875_575.844970703125]_[1106.6800537109375_221.8979949951172_565.8419799804688]_[1108.6700439453125_283.7229919433594_292.98602294921875]_[1071.72998046875_617.6190185546875_160.2979736328125]_[1055.0899658203125_608.2420043945312_995.5590209960938]_[1192.6099853515625_344.87298583984375_445.614013671875]_[1187.6800537109375_344.87298583984375_699.77001953125]_[993.7529907226562_347.2449951171875_346.4940185546875]_[981.4149780273438_347.2449951171875_783.2479858398438]_[969.166015625_513.8480224609375_771.385009765625]_[980.9520263671875_446.0069885253906_366.71002197265625]_[882.6710205078125_288.2760009765625_323.416015625]_[889.0819702148438_297.093994140625_820.8250122070312]_[803.4619750976562_21.38640022277832_477.552978515625]_[813.301025390625_27.085899353027344_654.636962890625]_[907.7329711914062_113.98600006103516_534.4010009765625]_[874.052001953125_131.7899932861328_593.1680297851562]_[1181.72998046875_255.59300231933594_781.509033203125]_[1177.6700439453125_248.6510009765625_353.7239990234375]_[1045.97998046875_225.4530029296875_904.072021484375]_[905.8579711914062_480.2330017089844_829.2219848632812]_[827.6610107421875_343.0589904785156_56.6400146484375]]}}_'navigation':{'pose':{'position':{'voxelSize':[10000_10000_10000]_'voxelCoordinates':[827.6610107421875_343.0589904785156_232.63999938964844]}}_'zoomFactor':25703.3403895381}_'perspectiveOrientation':[-0.5007818341255188_-0.4948247969150543_-0.48889824748039246_0.5151069760322571]_'perspectiveZoom':252192.47527422346_'layout':'4panel'}"
control9_link = "https://viz.boss.neurodata.io/#!{'layers':{'Ch1':{'type':'image'_'source':'boss://https://api.boss.neurodata.io/ailey-dev/Thy1eYFP_Control_9_upload20170919/Ch1?window=0,10000'_'opacity':0.5_'max':0.22}_'fiducials':{'type':'pointAnnotation'_'points':[[1022.3200073242188_1622.8199462890625_1078.8800048828125]_[1022.3200073242188_693.6110229492188_866.1690063476562]_[625.031005859375_1337.9300537109375_419.3179931640625]_[1379.0400390625_1250.4599609375_407.3059997558594]_[1042.3299560546875_2020.6800537109375_307.3059997558594]_[1731.3299560546875_1535.3699951171875_727.5330200195312]_[365.6629943847656_1565.31005859375_743.5800170898438]_[1005.739990234375_697.1519775390625_860.3709716796875]_[748.9910278320312_681.9400024414062_1188.4599609375]_[1269.7099609375_686.9290161132812_1201.3199462890625]_[851.2659912109375_1938.8299560546875_212.99600219726562]_[1242.219970703125_1950.1400146484375_196.11399841308594]_[757.8770141601562_503.5_1022.6500244140625]_[1251.739990234375_483.5_1059.4000244140625]_[1222.4200439453125_593.5_1164.699951171875]_[792.5969848632812_623.5_1158.1500244140625]_[1051.9300537109375_663.5_841.2999877929688]_[939.6229858398438_673.5_852.7899780273438]_[1364.9300537109375_613.5_885.9290161132812]_[707.7949829101562_663.5_864.947021484375]_[1006.8200073242188_933.5_1064.06005859375]_[869.2540283203125_976.3319702148438_534.1220092773438]_[1190.239990234375_979.3209838867188_511.12701416015625]_[1058.449951171875_1059.3199462890625_527.197998046875]_[968.4550170898438_1059.3199462890625_530.4130249023438]_[413.760986328125_1255.3199462890625_556.1270141601562]_[1681.1099853515625_1247.3199462890625_589.18798828125]_[1038.25_1902.3199462890625_523.0659790039062]_[744.3729858398438_2062.320068359375_512.0449829101562]_[1225.5999755859375_2035.3199462890625_449.59600830078125]_[329.27099609375_2036.3199462890625_589.18798828125]_[1247.6400146484375_2036.3199462890625_1257.760009765625]_[784.781982421875_2036.3199462890625_1224.699951171875]_[1049.27001953125_2286.320068359375_482.6579895019531]_[1053.8199462890625_2059.0400390625_335.6579895019531]_[358.3340148925781_2045.4000244140625_521.6580200195312]_[223.33399963378906_2045.4000244140625_1010.22998046875]_[1856.18994140625_2034.4000244140625_1103.43994140625]_[801.0499877929688_2272.669921875_566.6580200195312]_[1276.530029296875_2269.909912109375_609.1829833984375]_[667.0059814453125_1876.489990234375_719.1829833984375]_[1435.9100341796875_1868.3199462890625_702.7479858398438]_[1438.260009765625_1868.3199462890625_855.6060180664062]_[658.8189697265625_1859.4200439453125_699.6950073242188]_[460.72100830078125_1582.7099609375_585.9569702148438]_[1619.3199462890625_1566.47998046875_585.9569702148438]_[697.7000122070312_1404.6500244140625_46.359798431396484]_[1343.3900146484375_1444.0400390625_27.424299240112305]_[994.3099975585938_1566.47998046875_225.95700073242188]_[1105.4200439453125_1571.6199951171875_225.95700073242188]_[1480.6700439453125_2235.18994140625_415.9570007324219]_[598.7329711914062_2192.43994140625_415.9570007324219]_[1735.3299560546875_1848.8599853515625_415.9570007324219]_[1565.25_1647.760009765625_865.9569702148438]_[43.448699951171875_1554.1300048828125_629.6489868164062]]}}_'navigation':{'pose':{'position':{'voxelSize':[5160_5160_5160]_'voxelCoordinates':[43.448699951171875_1554.1300048828125_629.6489868164062]}}_'zoomFactor':29268.868618995297}_'perspectiveOrientation':[0.01311181578785181_0.006101356819272041_0.011622224934399128_0.9998260736465454]_'perspectiveZoom':240303.3975210289_'layout':'4panel'}"

In [46]:
control9_pts = get_points_from_viz_link(control9_link, 'fiducials')
atlas_pts = get_points_from_viz_link(atlas_link, 'fiducials')

In [47]:
rfid = control9_pts['points'] * control9_pts['vox_size']
afid = atlas_pts['points'] * atlas_pts['vox_size']

#### rescale data to match daniel's range:
**PIR**
- x: [-6575, 6575]
- y: [-3975,3975]
- z: [-5675, 5675]

In [48]:
rfid[-1]

array([  224.19529175,  8019.3108252 ,  3248.98877197])

In [49]:
afid[-1]

array([ 8276.61010742,  3430.58990479,   566.40014648])

In [50]:
min_vals_a = np.array([-6575, -3975, -5675])
max_vals_a = np.array([6575, 3975, 5675])
max_vals_r = np.array(true_size_raw)[[2,0,1]]
min_vals_r = np.zeros((1,3))
max_vals_a2 = np.array(true_size_atlas)
min_vals_a2 = np.zeros((1,3))
# first rescale [0,1]
rfid_norm = (rfid - min_vals_r)/(max_vals_r - min_vals_r)
# then rescale to scales above
rfid_rescaled = min_vals_a[[2,0,1]]*(1.0 - rfid_norm) + max_vals_a[[2,0,1]]*rfid_norm
# now same for atlas fiducials
afid_norm = (afid - min_vals_a2)/(max_vals_a2 - min_vals_a2)
# same for rescaling
afid_rescaled = min_vals_a * (1.0 - afid_norm) + max_vals_a * afid_norm

In [51]:
rfid_rescaled[-1]

array([-5446.69317387,  1408.12873602,  -462.22144197])

In [52]:
afid_rescaled[-1]

array([ 1670.25931156,  -565.85128212, -5111.08406468])

In [53]:
afid_r = r._reorient_landmarks(afid_rescaled, atlas_fid_o, trans_o)
rfid_r = r._reorient_landmarks(rfid_rescaled, raw_fid_o, trans_o)

In [54]:
afid_r[-1]

array([ 1670.25931156,  -565.85128212, -5111.08406468])

In [55]:
rfid_r[-1]

array([ 1408.12873602,  -462.22144197, -5446.69317387])

In [56]:
np.mean(r._compute_error(afid_r, rfid_r))

882.04520122966301

In [57]:
rfid_rh = np.append(rfid_r, np.ones((rfid_r.shape[0],1)), axis=1)

In [58]:
rfid_rh[-1]

array([  1.40812874e+03,  -4.62221442e+02,  -5.44669317e+03,
         1.00000000e+00])

In [59]:
rfid_a2t = fid_apply_affine(rfid_rh, a2t)
rfid_t2a = fid_apply_affine(rfid_rh, t2a)

In [60]:
rfid_a2t[-1]

array([ 1034.75291869,  -194.17936713, -5915.95643855])

In [61]:
rfid_t2a[-1]

array([ 1813.19723456,  -701.22828805, -5008.46248259])

In [62]:
afid_r[-1]

array([ 1670.25931156,  -565.85128212, -5111.08406468])

In [63]:
np.mean(r._compute_error(afid_r, rfid_a2t))

1536.318132724841

In [64]:
np.mean(r._compute_error(afid_r, rfid_t2a))

450.15524887560514

In [68]:
xyz_h = np.zeros(phi.shape)

In [69]:
xyz_h[:,:,:,0] = sitk.GetArrayViewFromImage(x)
xyz_h[:,:,:,1] = sitk.GetArrayViewFromImage(y)
xyz_h[:,:,:,2] = sitk.GetArrayViewFromImage(z)

In [70]:
x_i = xyz_h[0,:,0,0]
y_i = xyz_h[0,0,:,1]
z_i = xyz_h[:,0,0,2]

In [74]:
# afid_lddmm = scipy.interpolate.interpn(xyz, phi, afid_c[:,[2,0,1]])
afid_lddmm_x = scipy.interpolate.interpn((z_i, x_i, y_i), phix_np, afid_r[:,[2,0,1]])
afid_lddmm_y = scipy.interpolate.interpn((z_i, x_i, y_i), phiy_np, afid_r[:,[2,0,1]])
afid_lddmm_z = scipy.interpolate.interpn((z_i, x_i, y_i), phiz_np, afid_r[:,[2,0,1]])

In [75]:
afid_lddmm = np.array([[i, j, k] for i,j,k in zip(afid_lddmm_x, afid_lddmm_y, afid_lddmm_z)])

In [83]:
afid_r[-1]

array([ 1670.25931156,  -565.85128212, -5111.08406468])

In [76]:
afid_lddmm[-1]

array([ 1610.59701002,  -640.53632873, -5074.51001107])

In [77]:
rfid_a2t[-1]

array([ 1034.75291869,  -194.17936713, -5915.95643855])

In [78]:
rfid_t2a[-1]

array([ 1813.19723456,  -701.22828805, -5008.46248259])

In [79]:
np.mean(r._compute_error(rfid_a2t, afid_lddmm))

1461.9049528223227

In [80]:
np.mean(r._compute_error(rfid_t2a, afid_lddmm))

373.63209541014544

In [276]:
r._compute_error(rfid_t2a, afid_lddmm)

array([[  66.09652932],
       [ 219.0672453 ],
       [ 786.67863578],
       [ 375.46435085],
       [ 203.92895486],
       [ 287.78621937],
       [ 388.24758279],
       [ 116.76222036],
       [ 367.71574061],
       [ 437.69876699],
       [ 206.0404683 ],
       [ 267.38220981],
       [ 399.21325358],
       [ 581.54545287],
       [ 346.95044267],
       [ 267.8280555 ],
       [ 184.41234466],
       [ 218.09571782],
       [ 378.65348687],
       [ 165.47428259],
       [  82.96872061],
       [ 131.22203526],
       [ 134.99625478],
       [ 257.0896286 ],
       [ 107.72542154],
       [ 259.43974287],
       [ 355.98781228],
       [ 323.4149485 ],
       [ 720.4704984 ],
       [ 939.96329361],
       [ 288.22681408],
       [ 525.93586695],
       [ 305.35322403],
       [ 321.08427664],
       [ 323.57115365],
       [ 655.56537581],
       [ 465.27968724],
       [ 479.90926761],
       [ 174.20995905],
       [ 230.68346104],
       [ 592.2567223 ],
       [ 524.440

In [94]:
afid_lddmm_h = np.append(afid_lddmm, np.ones((afid_lddmm.shape[0],1)), axis=1)

In [95]:
afid_lddmm_h[-1]

array([  1.61059701e+03,  -6.40536329e+02,  -5.07451001e+03,
         1.00000000e+00])

In [96]:
afid_lddmm_af1 = fid_apply_affine(afid_lddmm_h, t2a)
afid_lddmm_af2 = fid_apply_affine(afid_lddmm_h, a2t)

In [97]:
afid_lddmm_af1[-5]

array([ 5980.03525117, -1334.48197561,  2078.77533927])

In [98]:
afid_lddmm_af2[-5]

array([ 4735.5234815 , -1315.86251764,  2036.87295865])

In [99]:
rfid_r[-5]

array([ 4906.54208183, -1654.39778539,  2105.37268462])

In [260]:
np.mean(r._compute_error(rfid_r, afid_lddmm_af2))

381.79759452677854

In [251]:
afid_lddmm_af2[-1]

array([ 1211.93900588,  -384.34928162, -5519.44231012])

In [262]:
afid_lddmm_n = (afid_lddmm_af2 + max_vals_a)/np.array([13150.0, 7950.0, 11350.0])

In [263]:
afid_lddmm_n

array([[ 0.63128432,  0.76046095,  0.46872981],
       [ 0.27303424,  0.58730944,  0.45991871],
       [ 0.48288003,  0.30556415,  0.34431701],
       [ 0.48692226,  0.31576561,  0.61159665],
       [ 0.78645204,  0.23877022,  0.49011124],
       [ 0.58483198,  0.49921326,  0.78396217],
       [ 0.5905498 ,  0.4902711 ,  0.18092032],
       [ 0.26777101,  0.59112025,  0.46669032],
       [ 0.28543655,  0.80640819,  0.36115249],
       [ 0.28785462,  0.80047546,  0.57712323],
       [ 0.75706422,  0.16633919,  0.40862644],
       [ 0.75237392,  0.16490686,  0.5662393 ],
       [ 0.18627744,  0.67123365,  0.3438397 ],
       [ 0.19395957,  0.67210482,  0.59026344],
       [ 0.23842747,  0.77583723,  0.55954377],
       [ 0.22980673,  0.79004965,  0.37527738],
       [ 0.26163206,  0.5705814 ,  0.49539566],
       [ 0.26294505,  0.57127492,  0.43816958],
       [ 0.25029616,  0.57786865,  0.62662544],
       [ 0.24742127,  0.5997399 ,  0.33039213],
       [ 0.36141416,  0.75432418,  0.469

In [264]:
out_pts = r._reorient_landmarks(afid_lddmm_n - np.array([0.5,0.5,0.5]), trans_o, "LAI")

In [268]:
out_pts_vox = (out_pts + np.array([0.5,0.5,0.5])) * np.array([269, 319, 177])

In [269]:
out_pts_vox

array([[ 142.91168117,  117.62030285,  134.60158903],
       [ 145.281867  ,  231.90207632,  103.95377134],
       [ 176.37872309,  164.96127192,   54.08485461],
       [ 104.4805006 ,  163.67179923,   55.8905126 ],
       [ 137.16007603,   68.12179992,   42.26232852],
       [  58.11417535,  132.43859743,   88.36074642],
       [ 220.33243343,  130.61461495,   86.77798403],
       [ 143.46030329,  233.58104736,  104.62828477],
       [ 171.84998074,  227.94573896,  142.73424964],
       [ 113.75385217,  227.17437721,  141.68415659],
       [ 159.07948696,   77.49651524,   29.44203593],
       [ 116.68162741,   78.99272095,   29.18851409],
       [ 176.5071208 ,  259.57749564,  118.80835621],
       [ 110.21913496,  257.12689773,  118.96255277],
       [ 118.48272612,  242.94163732,  137.32319022],
       [ 168.05038433,  245.69165335,  139.83878881],
       [ 135.73856746,  235.53937351,  100.99290742],
       [ 151.13238353,  235.12052756,  101.11566148],
       [ 100.43775556,  239.

In [236]:
rfid

array([[  5275.17123779,   8373.75092285,   5567.0208252 ],
       [  5275.17123779,   3579.03287842,   4469.43207275],
       [  3225.15999023,   6903.71907715,   2163.68084473],
       [  7115.84660156,   6452.37339844,   2101.69895874],
       [  5378.42257324,  10426.70907715,   1585.69895874],
       [  8933.66257324,   7922.5091748 ,   3754.0703833 ],
       [  1886.82105103,   8076.99990234,   3836.87288818],
       [  5189.61834961,   3597.3042041 ,   4439.51421387],
       [  3864.79370361,   3518.8104126 ,   6132.45339844],
       [  6551.70339844,   3544.55372314,   6198.81092285],
       [  4392.53251465,  10004.36257324,   1099.05937134],
       [  6409.85504883,  10062.72247559,   1011.94823181],
       [  3910.64539307,   2598.06      ,   5276.87412598],
       [  6458.97834961,   2494.86      ,   5466.50412598],
       [  6307.68742676,   3062.46      ,   6009.85174805],
       [  4089.80044189,   3217.26      ,   5976.05412598],
       [  5427.95907715,   3423.66      

In [237]:
center_raw

array([ 6604.8,  3676.5,  5572.8])

In [238]:
rfid - center_raw[[2,0,1]]

array([[ -297.62876221,  1768.95092285,  1890.5208252 ],
       [ -297.62876221, -3025.76712158,   792.93207275],
       [-2347.64000977,   298.91907715, -1512.81915527],
       [ 1543.04660156,  -152.42660156, -1574.80104126],
       [ -194.37742676,  3821.90907715, -2090.80104126],
       [ 3360.86257324,  1317.7091748 ,    77.5703833 ],
       [-3685.97894897,  1472.19990234,   160.37288818],
       [ -383.18165039, -3007.4957959 ,   763.01421387],
       [-1708.00629639, -3085.9895874 ,  2455.95339844],
       [  978.90339844, -3060.24627686,  2522.31092285],
       [-1180.26748535,  3399.56257324, -2577.44062866],
       [  837.05504883,  3457.92247559, -2664.55176819],
       [-1662.15460693, -4006.74      ,  1600.37412598],
       [  886.17834961, -4109.94      ,  1790.00412598],
       [  734.88742676, -3542.34      ,  2333.35174805],
       [-1482.99955811, -3387.54      ,  2299.55412598],
       [ -144.84092285, -3181.14      ,   664.60793701],
       [ -724.34539307, -3129.5

In [242]:
rfid_t = r._reorient_landmarks(rfid - center_raw[[2,0,1]], raw_fid_o, "LAI")

In [243]:
rfid_rt = (rfid_t + center_raw[[2,0,1]])
rfid_rtn = rfid_rt/np.array(true_size_raw)[[2,0,1]]

In [244]:
rfid_rtn

array([[ 0.5267037 ,  0.36608596,  0.75710878],
       [ 0.5267037 ,  0.72905819,  0.6078379 ],
       [ 0.71063379,  0.47737107,  0.29425824],
       [ 0.36155554,  0.51153908,  0.28582877],
       [ 0.51743984,  0.21067185,  0.21565333],
       [ 0.19845835,  0.4002461 ,  0.51054949],
       [ 0.83071158,  0.38855076,  0.52181054],
       [ 0.53437963,  0.72767501,  0.6037691 ],
       [ 0.65324489,  0.73361719,  0.83400699],
       [ 0.41217131,  0.73166835,  0.84303154],
       [ 0.60589537,  0.24264455,  0.14947088],
       [ 0.42489816,  0.23822656,  0.13762386],
       [ 0.64913101,  0.80332031,  0.71764914],
       [ 0.42049075,  0.81113281,  0.74343861],
       [ 0.43406479,  0.76816406,  0.8173333 ],
       [ 0.63305695,  0.75644531,  0.81273686],
       [ 0.51299535,  0.74082031,  0.59038596],
       [ 0.56498936,  0.73691406,  0.59844911],
       [ 0.36808794,  0.76035156,  0.62170457],
       [ 0.67231714,  0.74082031,  0.60698037],
       [ 0.53387963,  0.63535156,  0.746

In [245]:
rfid_vox = rfid_rtn * np.array([269, 319, 177])

In [270]:
rfid_vox[-1]

array([ 263.58902764,  125.34083142,   78.20903205])

In [271]:
out_pts_vox[-1]

array([ 265.31321422,  130.10011081,   79.94278958])

In [280]:
out_pts_vox.astype('float16').tolist()

[[142.875, 117.625, 134.625],
 [145.25, 231.875, 103.9375],
 [176.375, 165.0, 54.09375],
 [104.5, 163.625, 55.875],
 [137.125, 68.125, 42.25],
 [58.125, 132.5, 88.375],
 [220.375, 130.625, 86.75],
 [143.5, 233.625, 104.625],
 [171.875, 228.0, 142.75],
 [113.75, 227.125, 141.625],
 [159.125, 77.5, 29.4375],
 [116.6875, 79.0, 29.1875],
 [176.5, 259.5, 118.8125],
 [110.25, 257.25, 118.9375],
 [118.5, 243.0, 137.375],
 [168.0, 245.75, 139.875],
 [135.75, 235.5, 101.0],
 [151.125, 235.125, 101.125],
 [100.4375, 239.125, 102.3125],
 [180.125, 240.125, 106.125],
 [142.625, 203.75, 133.5],
 [160.125, 194.875, 64.5625],
 [123.4375, 196.125, 65.1875],
 [133.75, 186.0, 70.25],
 [150.375, 188.25, 67.0625],
 [211.375, 161.625, 70.625],
 [67.4375, 163.875, 77.0],
 [137.625, 80.0, 58.25],
 [178.375, 61.90625, 79.5625],
 [103.1875, 65.4375, 73.1875],
 [234.875, 63.4375, 74.125],
 [104.875, 69.3125, 148.25],
 [177.125, 67.875, 148.5],
 [136.875, 36.34375, 66.6875],
 [139.125, 60.75, 48.65625],
 [208.62

## add transformed fiducials to image

In [281]:
link = "https://viz.boss.neurodata.io/#!{'layers':{'control9,res3,rescaled':{'type':'image'_'source':'boss://https://api.boss.neurodata.io/control9_to_ara/daniel_reg/control9_res3_rescaled'_'opacity':0.5_'color':2}_'atlas,to,control9':{'type':'image'_'source':'boss://https://api.boss.neurodata.io/control9_to_ara/daniel_reg/atlas_to_control9'_'opacity':0.5_'color':5}}_'navigation':{'pose':{'position':{'voxelSize':[41280_41280_41280]_'voxelCoordinates':[248.32693481445312_125.8307876586914_80.39444732666016]}}_'zoomFactor':41280}_'perspectiveOrientation':[0.0009365863515995443_-0.008762896060943604_0.9997307658195496_0.021431967616081238]_'perspectiveZoom':377107.4582453634_'layout':'4panel'}"

In [282]:
link_a = add_points_layer_to_viz_link(link, 'atlas_fiducials', points=out_pts_vox.tolist())
link_r = add_points_layer_to_viz_link(link, 'raw_fiducials', points=rfid_vox.astype('float16').tolist())

replaced commas
source
replaced commas
replaced commas
source
replaced commas


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [283]:
link_ar = add_points_layer_to_viz_link(link_r, 'atlas_fiducials', points=out_pts_vox.astype('float16').tolist())

replaced commas
source
source
replaced commas


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [286]:
link_a2 = add_points_layer_to_viz_link(link_a, 'raw_fiducials', points=rfid_vox.tolist())

replaced commas
source
replaced commas
source


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
